## Finger Detection Python

##### Install Requirement Pakages

In [34]:
# !pip install -r requirements.txt

##### Import Nesscessary Pakages

In [35]:
from mediapipe import solutions
import cv2
import numpy as np

##### Define Constant

In [36]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green

# Define indices for fingertips and key landmarks
FINGER_TIPS = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky

##### Function for check the finger is open

In [37]:
def is_finger_extended(hand_landmarks, tip_idx, lower_idx):
    """
    Check if a finger is extended by comparing the y-coordinate (height)
    of the fingertip and its lower joint.
    """
    # Print the y-coordinates of the fingertip and lower joint
    print(f"tip_idx.y ({tip_idx}): {hand_landmarks[tip_idx].y}")
    print(f"lower_idx.y ({lower_idx}): {hand_landmarks[lower_idx].y}")
    
    return hand_landmarks[tip_idx].y < hand_landmarks[lower_idx].y


##### Function for count finger

In [38]:
def count_fingers(hand_landmarks, handedness_label):
    """
    Count the number of extended fingers for a detected hand.
    """
    extended_fingers = 0

    # Check non-thumb fingers
    for tip_idx in FINGER_TIPS[1:]:  # Skip the thumb
        lower_joint_idx = tip_idx - 2
        if is_finger_extended(hand_landmarks, tip_idx, lower_joint_idx):
            extended_fingers += 1

    # Thumb detection
    if handedness_label == "Right":
        thumb_extended = hand_landmarks[FINGER_TIPS[0]].x < hand_landmarks[FINGER_TIPS[0] - 1].x
    else:  # Left hand
        thumb_extended = hand_landmarks[FINGER_TIPS[0]].x > hand_landmarks[FINGER_TIPS[0] - 1].x

    if thumb_extended:
        extended_fingers += 1

    return extended_fingers

##### Function for draw land mark count finger

In [39]:
def draw_landmarks_and_count_fingers(rgb_image, results):
    """
    Draw hand landmarks and count the number of extended fingers.
    """
    annotated_image = np.copy(rgb_image)
    height, width, _ = annotated_image.shape

    for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
        handedness_label = results.multi_handedness[idx].classification[0].label

        # Draw hand landmarks
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        # Count extended fingers
        finger_count = count_fingers(hand_landmarks.landmark, handedness_label)

        # Get bounding box for text placement
        x_coordinates = [lm.x for lm in hand_landmarks.landmark]
        y_coordinates = [lm.y for lm in hand_landmarks.landmark]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Display handedness and finger count
        cv2.putText(annotated_image,
                    f"{handedness_label}: {finger_count} fingers",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

##### Initialize Mediapipe Hands

In [40]:
mp_hands = solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

I0000 00:00:1734327641.025912   95047 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1734327641.027406  102653 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics (TGL GT1)


##### Using Video for Detection

In [41]:
print("Press 'q' to exit.")
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip frame horizontally for mirror effect
    frame = cv2.flip(frame, 1)

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    results = hands.process(rgb_frame)

    # Annotate frame if hands are detected
    if results.multi_hand_landmarks:
        frame = draw_landmarks_and_count_fingers(frame, results)

    # Display annotated frame
    cv2.imshow('Finger Count', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

W0000 00:00:1734327641.062922  102639 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734327641.093683  102643 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Press 'q' to exit.
tip_idx.y (8): 0.32143107056617737
lower_idx.y (6): 0.5154690742492676
tip_idx.y (12): 0.29915332794189453
lower_idx.y (10): 0.5147393941879272
tip_idx.y (16): 0.6135349869728088
lower_idx.y (14): 0.627799928188324
tip_idx.y (20): 0.6856539249420166
lower_idx.y (18): 0.6996999382972717
tip_idx.y (8): 0.6819371581077576
lower_idx.y (6): 0.626457929611206
tip_idx.y (12): 0.29551443457603455
lower_idx.y (10): 0.5137374401092529
tip_idx.y (16): 0.28803750872612
lower_idx.y (14): 0.5014408230781555
tip_idx.y (20): 0.38547223806381226
lower_idx.y (18): 0.5673631429672241
tip_idx.y (8): 0.6867643594741821
lower_idx.y (6): 0.6146634817123413
tip_idx.y (12): 0.29520347714424133
lower_idx.y (10): 0.5224553942680359
tip_idx.y (16): 0.2883971631526947
lower_idx.y (14): 0.5147512555122375
tip_idx.y (20): 0.37117722630500793
lower_idx.y (18): 0.5768042206764221
tip_idx.y (8): 0.6927459239959717
lower_idx.y (6): 0.6233107447624207
tip_idx.y (12): 0.3112180829048157
lower_idx.y (10)